In [1]:
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df= pd.read_csv('/content/drive/MyDrive/Capstone/Curated_dataset/Dataset1.csv')

In [4]:
df

,path,Text,class
0,/content/drive/MyDrive/Capstone/Curated_datase...,আসলে বুঝাবো,উত্তেজিত
1,/content/drive/MyDrive/Capstone/Curated_datase...,রাজশাহীতে নেবে দুলাভাই আর আপুর বাসায় উঠবো,উত্তেজিত
2,/content/drive/MyDrive/Capstone/Curated_datase...,আমি তোমার জন্য পাগল,উত্তেজিত
3,/content/drive/MyDrive/Capstone/Curated_datase...,তোর সঙ্গে হাজার বছর আমার জীবনটা কাটাতে চাই,উত্তেজিত
4,/content/drive/MyDrive/Capstone/Curated_datase...,তুমি আমাকে পছন্দ করো তুমি আমাকে ভালোবাসো,উত্তেজিত
...,...,...,...
928,/content/drive/MyDrive/Capstone/Curated_datase...,আমি আপনাকে নিষেধ করেছি আপনি কি আসবেন না বেরিয়...,রাগান্বিত
929,/content/drive/MyDrive/Capstone/Curated_datase...,এই কি বলছে থাপ্পড় আবি বলার মত কেউ নাই,রাগান্বিত
930,/content/drive/MyDrive/Capstone/Curated_datase...,Google Speech Recognition could not understand...,রাগান্বিত
931,/content/drive/MyDrive/Capstone/Curated_datase...,খবরদার আমাকে ফলো করবেন না আমি যেন আপনাকে না দে...,রাগান্বিত


In [5]:
 
import csv
with open('/content/drive/MyDrive/Capstone/Curated_dataset/Dataset1.csv', encoding='utf-8') as f:
    datastore = csv.DictReader(f)
    path=[]
    sentences = []
    labels = []

    for item in datastore:
        path.append(item['path'])
        sentences.append(item['Text'])
        labels.append(item['class'])

In [6]:
set(labels)

{'উত্তেজিত', 'খুশি', 'দুঃখিত', 'নিরপেক্ষ', 'রাগান্বিত'}

In [7]:
categories = list(set(labels))

In [8]:

categories

['খুশি', 'উত্তেজিত', 'দুঃখিত', 'নিরপেক্ষ', 'রাগান্বিত']

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test,z_train,z_test = train_test_split(
  path, sentences, labels, test_size=0.2, random_state=42)

In [10]:

!pip install ktrain

In [11]:
import ktrain
from ktrain import text

In [12]:
 MODEL_NAME = 'sagorsarker/bangla-bert-base'
 t = text.Transformer(MODEL_NAME, maxlen=500)

In [13]:
trn = t.preprocess_train( y_train,z_train)
val = t.preprocess_test( y_test,z_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 6
	95percentile : 12
	99percentile : 15


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 6
	95percentile : 11
	99percentile : 13


In [14]:
model = t.get_classifier()


In [15]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [16]:
preproc = t
# save model and Preprocessor instance after partially training
ktrain.get_predictor(model, preproc).save('/tmp/my_predictor')

In [ ]:

%matplotlib inline
learner.lr_find(show_plot=True, max_epochs=1)

In [18]:
learner.fit_onecycle(8e-5, 10)



begin training using onecycle policy with max lr of 8e-05...
Epoch 1/10
125/125 [==============================] - 106s 741ms/step - loss: 1.6203 - accuracy: 0.2392 - val_loss: 1.5258 - val_accuracy: 0.2995
Epoch 2/10
125/125 [==============================] - 93s 746ms/step - loss: 1.3621 - accuracy: 0.4572 - val_loss: 1.6632 - val_accuracy: 0.3369
Epoch 3/10
125/125 [==============================] - 96s 771ms/step - loss: 1.0252 - accuracy: 0.6182 - val_loss: 1.6996 - val_accuracy: 0.3476
Epoch 4/10
125/125 [==============================] - 97s 772ms/step - loss: 0.7028 - accuracy: 0.7604 - val_loss: 2.0073 - val_accuracy: 0.3155
Epoch 5/10
125/125 [==============================] - 96s 771ms/step - loss: 0.6403 - accuracy: 0.7746 - val_loss: 1.9425 - val_accuracy: 0.3209
Epoch 6/10
125/125 [==============================] - 97s 771ms/step - loss: 0.3865 - accuracy: 0.8630 - val_loss: 2.2109 - val_accuracy: 0.3369
Epoch 7/10
125/125 [==============================] - 96s 771ms/st

In [19]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

    উত্তেজিত       0.29      0.40      0.34        30
        খুশি       0.50      0.31      0.38        42
      দুঃখিত       0.27      0.29      0.28        34
    নিরপেক্ষ       0.28      0.31      0.29        36
   রাগান্বিত       0.34      0.33      0.34        45

    accuracy                           0.33       187
   macro avg       0.34      0.33      0.33       187
weighted avg       0.34      0.33      0.33       187



array([[12,  4,  5,  3,  6],
       [12, 13,  8,  4,  5],
       [ 4,  3, 10,  9,  8],
       [ 6,  3,  6, 11, 10],
       [ 7,  3,  8, 12, 15]])

In [22]:
learner.view_top_losses(n=1, preproc=t)

----------
id:182 | loss:7.6 | true:উত্তেজিত | pred:খুশি)



In [20]:

predictor = ktrain.get_predictor(learner.model, preproc=t)

In [21]:
predictor.save('/content/drive/MyDrive/Capstone/mymodel')

In [22]:
learner.model.save_pretrained('/content/drive/MyDrive/Capstone/myTmodel')

In [23]:

# reload the model using transformers directly
from transformers import *
model = TFAutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Capstone/mymodel')
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [24]:
learner.model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  164396544 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3845      
Total params: 164,400,389
Trainable params: 164,400,389
Non-trainable params: 0
_________________________________________________________________


In [25]:
!pip install SpeechRecognition

     |████████████████████████████████| 32.8MB 99kB/s 


In [26]:
import speech_recognition as sr
p='/content/drive/MyDrive/Capstone/Curated_dataset/খুশি/H-001.flac'
r=sr.Recognizer()
with sr.AudioFile(p) as source:
              audio=r.record(source)
              try:
               text=r.recognize_google(audio, language='bn_BD')
              except sr.UnknownValueError:
               text = "Google Speech Recognition could not understand audio"

In [27]:
predictor.predict(text)

'খুশি'

In [28]:
predictor.predict_proba(text)

array([3.0188973e-03, 9.9303108e-01, 3.4867975e-04, 8.1707363e-04,
       2.7843220e-03], dtype=float32)